# Plotting routine for the June2 2022 Event - PSP

In [1]:
test = 'test'

In [2]:
%load_ext autoreload

%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pickle as p

import datetime as datetime
from datetime import timedelta
import seaborn as sns
import pandas as pd
import os
#import copy as cp

import multiprocess as mp
import py3dcore as py3dcore

import py3dcore.fluxplot as fp

plt.switch_backend('agg')


import warnings
warnings.filterwarnings('ignore')

import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")
#logging.getLogger("sunpy").setLevel("WARNING")
#logging.getLogger("py3dcore_h4c.fluxplot").setLevel("WARNING")

#print(sns.__version__)

#### setting the times for py3DCORE

In [3]:
t_launch = datetime.datetime(2022, 6, 2, 7, 30, tzinfo=datetime.timezone.utc)

t_s = datetime.datetime(2022, 6, 6, 16, 30, tzinfo=datetime.timezone.utc) 
t_e = datetime.datetime(2022, 6, 7, 4, tzinfo=datetime.timezone.utc)

t_fit = [
       #datetime.datetime(2022, 6, 6, 16, 30, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 6, 18, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 6, 23, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 7, 2, tzinfo=datetime.timezone.utc)#,
        #datetime.datetime(2022, 6, 7, 8, tzinfo=datetime.timezone.utc)
     ]


In [4]:
fittingPoints = []

fittingPoints = [t_fit[i].strftime("%m/%d/%Y, %H:%M:%S") for i in range(len(t_fit))]
print(fittingPoints)

['06/06/2022, 18:00:00', '06/06/2022, 23:00:00', '06/07/2022, 02:00:00']


#### loading the pickle files

In [5]:
path = 'output/wind06062022_512_heeq_2/'

filepath = fp.loadpickle(path, number=-1)

In [6]:
filepath[:-7]

'output/wind06062022_512_heeq_2/04'

#### extracting parameters

In [7]:
res_mineps, res_mean, res_std, ind, keys, allres = fp.get_params(filepath, give_mineps=True)

print(ind)

INFO:py3dcore.fluxplot:Retrieved the following parameters for the run with minimum epsilon:
INFO:py3dcore.fluxplot: --cme_longitude 38.56
INFO:py3dcore.fluxplot: --cme_latitude 24.76
INFO:py3dcore.fluxplot: --cme_inclination 190.19
INFO:py3dcore.fluxplot: --cme_diameter_1au 0.31
INFO:py3dcore.fluxplot: --cme_aspect_ratio 1.55
INFO:py3dcore.fluxplot: --cme_launch_radius 11.27
INFO:py3dcore.fluxplot: --cme_launch_velocity 722.93
INFO:py3dcore.fluxplot: --t_factor -3.97
INFO:py3dcore.fluxplot: --cme_expansion_rate 1.14
INFO:py3dcore.fluxplot: --magnetic_decay_rate 1.64
INFO:py3dcore.fluxplot: --magnetic_field_strength_1au 6.52
INFO:py3dcore.fluxplot: --background_drag 2.04
INFO:py3dcore.fluxplot: --background_velocity 565.00


Results :

	cme_longitude: 		34.03 +/- 10.24
	cme_latitude: 		7.31 +/- 10.85
	cme_inclination: 		189.38 +/- 11.66
	cme_diameter_1au: 		0.10 +/- 0.04
	cme_aspect_ratio: 		2.82 +/- 0.75
	cme_launch_radius: 		17.23 +/- 4.17
	cme_launch_velocity: 		878.88 +/- 312.59
	t_factor: 		-70.90 +/- 121.31
	cme_expansion_rate: 		1.14 +/- 0.00
	magnetic_decay_rate: 		1.64 +/- 0.00
	magnetic_field_strength_1au: 		6.84 +/- 1.06
	background_drag: 		1.70 +/- 0.72
	background_velocity: 		428.21 +/- 77.25
490


In [8]:
out = fp.get_ensemble_stats(filepath)
out.describe()

,lon,lat,inc,D1AU,delta,launch radius,launch speed,t factor,expansion rate,B decay rate,B1AU,gamma,vsw
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.00,512.00,512.000000,512.000000,512.000000
mean,34.033859,7.309281,189.375885,0.101246,2.816705,17.234638,878.882080,-70.902878,1.14,1.64,6.838452,1.701151,428.215088
std,10.253194,10.859738,11.676010,0.041670,0.747044,4.170987,312.896881,121.433014,0.00,0.00,1.056572,0.720973,77.322342
min,-8.708972,-21.349159,150.687897,0.057344,1.042003,10.011787,401.274200,-249.909637,1.14,1.64,5.003109,0.202350,195.676895
25%,27.604721,-0.056215,181.423943,0.076212,2.284801,13.935882,600.883728,-169.342541,1.14,1.64,6.044040,1.143238,374.798004
50%,34.912729,7.175347,188.970535,0.087246,2.918592,16.961381,854.854828,-88.938419,1.14,1.64,6.751946,1.720093,417.167267
75%,40.708462,14.835738,198.464367,0.110912,3.440063,20.810674,1129.226318,12.507318,1.14,1.64,7.548338,2.294414,472.339401
max,58.096394,42.888878,221.469620,0.347151,3.991748,24.954008,1496.524902,244.740067,1.14,1.64,10.316495,2.999679,695.870605


In [9]:
# write output of describe() to csv-file
out.describe().to_csv(filepath+'parameters.csv')

# write parameters of whole ensemble to txt-file
with open(filepath+'parameters_ensemble.txt', 'a') as f:
    dfAsString = out.to_string(header=True, index=True)
    f.write(dfAsString)

In [10]:
# write output of describe() to txt-file    
with open(filepath+'parameters_summary.txt', 'a') as f:
    f.write('Summary of results')
    f.write('\n\n')
    f.write('Results:')
    f.write('\n')
    for i in range(1, len(keys)):    
        f.write("\t{}: \t\t{:.02f} +/- {:.02f}".format(keys[i], res_mean[i], res_std[i]))
        f.write('\n')
    f.write('\n')
    f.write('Parameters for run with min(eps):')
    f.write('\n')
    for i in range(1, len(keys)):
        f.write(" --{} {:.2f}".format(keys[i], res_mineps[i]))
        f.write('\n')
    f.write('\n')
    f.write('Fitting points:')
    f.write(str(fittingPoints))
    f.write('\n')

#### loading and plotting the fitting results

In [11]:
t_s = datetime.datetime(2022, 6, 6, 13, tzinfo=datetime.timezone.utc) 
t_e = datetime.datetime(2022, 6, 7, 13, tzinfo=datetime.timezone.utc)

In [12]:
fp.fullinsitu(observer='WIND', t_fit=t_fit, start=t_s, end=t_e, filepath=filepath, ref_frame='HEEQ', best=False, 
              ensemble=True, save_fig=True, legend=True, title=False, fit_points=True)

INFO:py3dcore.fluxplot:Using HelioSat to retrieve observer data


2022-06-06 13:00:00+00:00 2022-06-07 13:00:00+00:00


INFO:heliosat.datafile:fetch "https://cdaweb.gsfc.nasa.gov/pub/data/wind/mfi/mfi_h0/2022/wi_h0_mfi_20220606_v05.cdf"
INFO:heliosat.datafile:fetch "https://cdaweb.gsfc.nasa.gov/pub/data/wind/mfi/mfi_h0/2022/wi_h0_mfi_20220607_v05.cdf"
INFO:heliosat.datafile:fetch "https://cdaweb.gsfc.nasa.gov/pub/data/wind/orbit/pre_or/2022/wi_or_pre_20220607_v09.cdf"
INFO:heliosat.datafile:fetch "https://cdaweb.gsfc.nasa.gov/pub/data/wind/orbit/pre_or/2022/wi_or_pre_20220606_v09.cdf"


IndexError: index 143 is out of bounds for axis 0 with size 143

In [ ]:
fp.scatterparams(filepath)

In [ ]:
print(out['lon'].mean())
print(out['lon'].std())
print(out['launch radius'].std())
print(out['launch speed'].std())

In [ ]:
print(res_mineps[1])

#### Plotting the fluxrope and spacecraft positions

In [ ]:
#measurement times 
#tm0 =  datetime.datetime(2022, 6, 2, 6)
tm1 =  t_launch + datetime.timedelta(hours=5)
#tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d(spacecraftlist=['psp'], planetlist=['Earth'], t=tm1, filepath=filepath, legend=False, view_azim=0, title=False,
                view_elev=0, view_radius=0.058, index=0)

### make fluxrope 3d movie

In [ ]:
def make_frame(k):
    tm0 = t_launch
    t = tm0 + k*datetime.timedelta(hours=0.5)
    
    fig = fp.full3d(spacecraftlist=['psp'], planetlist=[], legend=False, title=False,
              t=t, filepath=filepath, save_fig=True, view_radius=0.3)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d/' + frmstr + '.jpg', dpi=300)  
    return fig
    

In [ ]:
inn = [i for i in range(100)]


mpool = mp.Pool(processes=4)
mpool.map(make_frame, inn[0:40])
mpool.close()
mpool.join()

print('done 1/4')

mpool = mp.Pool(processes=4)
mpool.map(make_frame, inn[40:80])
mpool.close()
mpool.join()

print('done 2/4')

#mpool = mp.Pool(processes=4)
#mpool.map(make_frame, inn[40:60])
#mpool.close()
#mpool.join()

#print('done 3/4')

#mpool = mp.Pool(processes=4)
#mpool.map(make_frame, inn[60:80])
#mpool.close()
#mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_movie.mp4 -y -loglevel quiet')

### plot fluxrope 3d multiview

In [ ]:
fig = fp.full3d_multiview(tm1, filepath=filepath)

### make fluxrope 3d multiview movie

In [ ]:
def make_frame2(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    frametime = k
    
    fig = fp.full3d_multiview_movie(t_launch = tm0, t = t, filepath=filepath,
                                    frametime=k)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d_multiview/'+frmstr+'.jpg',dpi=200)  
    return fig

In [ ]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

In [ ]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d_multiview/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_multiview_movie.mp4 -y -loglevel quiet')

# Used for Debug

In [ ]:
TP_A =  t_launch + datetime.timedelta(hours=2)
TP_B =  t_launch + datetime.timedelta(hours=40)


C_A = "xkcd:red"
C_B = "xkcd:blue"
    
sns.set_style('whitegrid')

fig = plt.figure(figsize=(15, 11),dpi=100)

ax1 = plt.subplot2grid((2, 3), (0, 0),rowspan=2,colspan=2,projection='3d')  
ax2 = plt.subplot2grid((2, 3), (0, 2),projection='3d')  
ax3 = plt.subplot2grid((2, 3), (1, 2),projection='3d')  

model_obj = fp.returnmodel(filepath)


fp.plot_configure(ax1, view_azim=150, view_elev=25, view_radius=.2,light_source=True) #view_radius=.08

fp.plot_3dcore(ax1, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax1, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax1,0.31,-0.25,0.0,-0.2)


fp.plot_configure(ax2, view_azim=165-90, view_elev=90, view_radius=.08,light_source=True)
    
fp.plot_3dcore(ax2, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax2, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax2,0.26,-0.41,0.08,0.0)



fp.plot_configure(ax3, view_azim=65, view_elev=-5, view_radius=.01,light_source=True)
    
fp.plot_3dcore(ax3, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax3, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax3,0.26,-0.41,0.08,0.0)

